In [ ]:
import pandas as pd

In [ ]:
col_used = [
'Make',
'Model',
'Year',
'Engine HP',
'Engine Cylinders',
'Transmission Type',
'Vehicle Style',
'highway MPG',
'city mpg',
'MSRP'
]

In [ ]:
df = pd.read_csv('data.csv')
df_fil = df[col_used]
df_fil.columns = df_fil.columns.str.replace(' ', '_').str.lower()
df_fil.rename({'msrp':'price'},axis=1,inplace=True)

In [ ]:
# Q1
df_fil.groupby('transmission_type').count()

In [ ]:
# Q2
import seaborn as sns
df_fil.dtypes[df_fil.dtypes != 'object'].index
df_cor = df_fil[df_fil.dtypes[df_fil.dtypes != 'object'].index]
sns.heatmap(df_cor.corr(),annot=True)

In [ ]:
from sklearn.model_selection import train_test_split
avg_price = df_fil.price.mean()
df_fil['above_average'] = (df_fil.price>avg_price).astype('int')




In [ ]:
x_train

In [ ]:
# Q3
from sklearn.metrics import mutual_info_score

df_split = df_fil.drop('price',axis=1)
train,_ = train_test_split(df_split.fillna(0),test_size=0.4,random_state=42)
test,val = train_test_split(_,test_size=0.5,random_state=42)

x_train = train.drop('above_average',axis=1)
y_train = train.above_average

x_test = test.drop('above_average',axis=1)
y_test = test.above_average

x_val = val.drop('above_average',axis=1)
y_val = val.above_average

cat_col = x_train.columns
score_para = {}
for i in cat_col:
    print(i)
    score = mutual_info_score(y_train,x_train[i]).round(2)
    print(score)
    print('----')
    score_para[i] = score



In [ ]:
# Q4
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score
dv = DictVectorizer()
x_train= dv.fit_transform(x_train.to_dict(orient='records'))
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(x_train,y_train)
x_val = dv.transform(x_val.to_dict(orient='records'))
pred = model.predict(x_val)
acc_main = accuracy_score(y_val,pred)
acc_main

In [ ]:
list(score_para.keys())

In [ ]:
# Q5
df_split = df_fil.drop('price',axis=1).fillna(0)

score_drop = []
for para in score_para.keys():
    train,_ = train_test_split(df_split.drop(para,axis=1),test_size=0.4,random_state=42)
    test,val = train_test_split(_,test_size=0.5,random_state=42)

    x_train = train.drop('above_average',axis=1)
    y_train = train.above_average

    x_test = test.drop('above_average',axis=1)
    y_test = test.above_average

    x_val = val.drop('above_average',axis=1)
    y_val = val.above_average


    dv = DictVectorizer()
    x_train= dv.fit_transform(x_train.to_dict(orient='records'))
    model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    model.fit(x_train,y_train)
    x_val = dv.transform(x_val.to_dict(orient='records'))
    pred = model.predict(x_val)
    acc = accuracy_score(y_val,pred)

    score_drop.append([para,acc_main - acc])
pd.DataFrame(score_drop,columns=['para','acc']).sort_values('acc')

In [159]:
from sklearn.linear_model import RidgeClassifier

train,_ = train_test_split(df_split,test_size=0.4,random_state=42)
test,val = train_test_split(_,test_size=0.5,random_state=42)

x_train = train.drop('above_average',axis=1)
y_train = train.above_average

x_test = test.drop('above_average',axis=1)
y_test = test.above_average

x_val = val.drop('above_average',axis=1)
y_val = val.above_average

dv = DictVectorizer()
x_train= dv.fit_transform(x_train.to_dict(orient='records'))
x_val = dv.transform(x_val.to_dict(orient='records'))
for a in [0, 0.01, 0.1, 1, 10]:   
    model = RidgeClassifier(solver='sag',alpha=a, random_state=42)
    model.fit(x_train,y_train)
    pred = model.predict(x_val)
    acc = accuracy_score(y_val,pred)
    print(a)
    print(acc.round(3))
    print('----')


0
0.94
----
0.01
0.941
----
0.1
0.94
----
1
0.94
----
10
0.935
----
